In [1]:
!nvidia-smi

Tue Feb  8 16:37:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow

In [4]:
import gc
import tensorflow as tf
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
gc.collect()

0

In [5]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_df = pd.read_csv('mami_training.csv', sep='\t', lineterminator='\r')
test_df = pd.read_csv('Test.csv', sep='\t', lineterminator='\n')
test_labels = pd.read_csv('test_labels.txt',header=None, sep='\t', lineterminator='\n')
X = train_df['Text Transcription'].copy()
for i in range(len(X)):
  X[i] = X[i].lower()
X_test_model = test_df['Text Transcription'].to_numpy()
for i in range(len(X_test_model)):
  X_test_model[i] = X_test_model[i].lower()
Y = train_df['misogynous']
X_train_model = X[:].to_numpy()
Y_train_model = Y[:].to_numpy()
print(X_train_model.shape, Y_train_model.shape)
print(Y_train_model.sum())

(10000,) (10000,)
5000


In [6]:
MAX_LENGTH = 50
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=MAX_LENGTH):
    features = [] 
    for e in examples:
        input_dict = tokenizer.encode_plus(
            e[3],
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e[1]
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [7]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
def try_model():
  clf = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
  print(clf.summary())
  clf.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-6, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  train_bert = pd.DataFrame({
  'guid': range(len(X_train_model)),
  'label': Y_train_model,
  'text_b': ['a']*len(X_train_model),
  'text_a': X_train_model
  })
  bert_train, bert_val = train_test_split(train_bert, test_size = 0.1)
  bert_train, bert_val = bert_train.to_numpy(), bert_val.to_numpy()
  bert_train = convert_examples_to_tf_dataset(bert_train, tokenizer)
  bert_train = bert_train.shuffle(2).batch(64).repeat(2)
  bert_val = convert_examples_to_tf_dataset(bert_val, tokenizer)
  bert_val = bert_val.shuffle(2).batch(64)
  clf.fit(
      bert_train, 
      epochs=5, 
      validation_data=bert_val, verbose = 2, 
      callbacks=[
              tf.keras.callbacks.EarlyStopping(
              monitor="val_accuracy",
              min_delta=0.005,
              patience=4,
              verbose=2,
              mode="max",
              baseline=None,
              restore_best_weights=True,
              ),
              GarbageCollectorCallback()])
  tf_batch = tokenizer(list(X_test_model), max_length=MAX_LENGTH, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = clf(tf_batch)
  label = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = np.array(tf.argmax(label, axis=1))
  return label

In [27]:
from sklearn.metrics import confusion_matrix
Y_test_model = test_labels[1].to_numpy()
Y_train_model = train_df['misogynous'].to_numpy()
print("######## Misogynous Prediction ########")
print("\nSubmission Score ")
y_pred = try_model()
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskA = f1_score(Y_test_model, y_pred, average='macro')
subtaskB = f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 208s - loss: 0.5976 - accuracy: 0.6746 - val_loss: 0.4887 - val_accuracy: 0.7820 - 208s/epoch - 738ms/step
Epoch 2/5
282/282 - 183s - loss: 0.4537 - accuracy: 0.8004 - val_loss: 0.4491 - val_accuracy: 0.8160 - 183s/epoch - 651ms/step
Epoch 3/5
282/282 - 183s - loss: 0.3775 - accuracy: 0.8446 - val_loss: 0.4377 - val_accuracy: 0.8100 - 183s/epoch - 650ms/step
Epoch 4/5
282/282 - 182s - loss: 0.3218 - accuracy: 0.8726 - val_loss: 0.4412 - val_accuracy: 0.8210 - 182s/epoch - 646ms/step
Epoch 5/5
282/282 - 183s - loss: 0.2683 - accuracy: 0.9007 - val_loss: 0.4683 - val_accuracy: 0.8140 - 183s/epoch - 649ms/step
######## Misogynous Prediction ########

Submission Score 
[[260 240]
 [ 90 410]] 
 F1 score - 0.6624040920716112


In [32]:
Y_train_model = train_df['shaming'].to_numpy()
Y_test_model = test_labels[2].to_numpy()
y_pred = try_model()
print("\n######## Shaming Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 208s - loss: 0.3942 - accuracy: 0.8683 - val_loss: 0.3447 - val_accuracy: 0.8780 - 208s/epoch - 737ms/step
Epoch 2/5
282/282 - 183s - loss: 0.3248 - accuracy: 0.8742 - val_loss: 0.3226 - val_accuracy: 0.8870 - 183s/epoch - 650ms/step
Epoch 3/5
282/282 - 183s - loss: 0.2738 - accuracy: 0.8894 - val_loss: 0.3245 - val_accuracy: 0.8940 - 183s/epoch - 650ms/step
Epoch 4/5
282/282 - 183s - loss: 0.2274 - accuracy: 0.9087 - val_loss: 0.3478 - val_accuracy: 0.8730 - 183s/epoch - 648ms/step
Epoch 5/5
282/282 - 183s - loss: 0.1738 - accuracy: 0.9329 - val_loss: 0.3857 - val_accuracy: 0.8760 - 183s/epoch - 650ms/step

######## Shaming Prediction ########
[[791  63]
 [ 98  48]] 
 F1 score -  0.6405856890597409


In [35]:
Y_train_model = train_df['stereotype'].to_numpy()
Y_test_model = test_labels[3].to_numpy()
y_pred = try_model()
print("\n######## Stereotype Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_227 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 207s - loss: 0.5740 - accuracy: 0.7049 - val_loss: 0.4842 - val_accuracy: 0.7740 - 207s/epoch - 733ms/step
Epoch 2/5
282/282 - 183s - loss: 0.4597 - accuracy: 0.7853 - val_loss: 0.4520 - val_accuracy: 0.7880 - 183s/epoch - 650ms/step
Epoch 3/5
282/282 - 184s - loss: 0.4008 - accuracy: 0.8181 - val_loss: 0.4616 - val_accuracy: 0.7760 - 184s/epoch - 651ms/step
Epoch 4/5
282/282 - 184s - loss: 0.3412 - accuracy: 0.8503 - val_loss: 0.5019 - val_accuracy: 0.7540 - 184s/epoch - 651ms/step
Epoch 5/5
282/282 - 184s - loss: 0.2784 - accuracy: 0.8856 - val_loss: 0.5402 - val_accuracy: 0.7570 - 184s/epoch - 652ms/step

######## Stereotype Prediction ########
[[438 212]
 [149 201]] 
 F1 score -  0.6175162714511391


In [38]:
Y_train_model = train_df['objectification'].to_numpy()
Y_test_model = test_labels[4].to_numpy()
y_pred = try_model()
print("\n######## Objectification Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_265 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 210s - loss: 0.5035 - accuracy: 0.7819 - val_loss: 0.4432 - val_accuracy: 0.8070 - 210s/epoch - 746ms/step
Epoch 2/5
282/282 - 184s - loss: 0.4152 - accuracy: 0.8164 - val_loss: 0.4163 - val_accuracy: 0.8240 - 184s/epoch - 653ms/step
Epoch 3/5
282/282 - 184s - loss: 0.3547 - accuracy: 0.8507 - val_loss: 0.4225 - val_accuracy: 0.8230 - 184s/epoch - 651ms/step
Epoch 4/5
282/282 - 183s - loss: 0.2956 - accuracy: 0.8836 - val_loss: 0.4506 - val_accuracy: 0.8180 - 183s/epoch - 651ms/step
Epoch 5/5
282/282 - 184s - loss: 0.2439 - accuracy: 0.9097 - val_loss: 0.4824 - val_accuracy: 0.8170 - 184s/epoch - 651ms/step

######## Objectification Prediction ########
[[568  84]
 [202 146]] 
 F1 score -  0.6520325678049824


In [41]:
Y_train_model = train_df['violence'].to_numpy()
Y_test_model = test_labels[5].to_numpy()
y_pred = try_model()
print("\n######## Violence Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_303 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 208s - loss: 0.3710 - accuracy: 0.8608 - val_loss: 0.3035 - val_accuracy: 0.9070 - 208s/epoch - 736ms/step
Epoch 2/5
282/282 - 184s - loss: 0.2890 - accuracy: 0.9055 - val_loss: 0.2879 - val_accuracy: 0.9070 - 184s/epoch - 652ms/step
Epoch 3/5
282/282 - 184s - loss: 0.2350 - accuracy: 0.9196 - val_loss: 0.2662 - val_accuracy: 0.9080 - 184s/epoch - 652ms/step
Epoch 4/5
282/282 - 184s - loss: 0.1865 - accuracy: 0.9369 - val_loss: 0.2733 - val_accuracy: 0.9150 - 184s/epoch - 654ms/step
Epoch 5/5
282/282 - 183s - loss: 0.1465 - accuracy: 0.9524 - val_loss: 0.2872 - val_accuracy: 0.9160 - 183s/epoch - 649ms/step

######## Violence Prediction ########
[[829  18]
 [110  43]] 
 F1 score -  0.6651003129218951


In [43]:
print("Subtask A score -", 
      subtaskA, 
      "\n Subtask B score - ", 
      subtaskB/(test_labels[5].sum()+ test_labels[4].sum()+ test_labels[3].sum()+test_labels[2].sum()+test_labels[1].sum()))

Subtask A score - 0.6624040920716112 
 Subtask B score -  0.6476459139075553
